In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
import datetime

# --- 1. 데이터 임포트 ---
from services.tables.Time_Attendance.daily_working_info_table import daily_work_info_df
from services.tables.HR_Core.position_info_table import position_info_df
from services.tables.HR_Core.position_table import position_df, position_order, grade_order

def create_figure():
    """
    제안 13-3: 직위별 워라밸 변화 추이 그래프를 생성합니다.
    """
    # --- 2. 데이터 준비 및 가공 ---
    daily_work_df = daily_work_info_df.copy()
    daily_work_df['DATE'] = pd.to_datetime(daily_work_df['DATE'])
    daily_work_df['PAY_PERIOD'] = daily_work_df['DATE'].dt.strftime('%Y-%m')

    # 날짜별 직위/직급 정보 추가
    pos_info_with_name = pd.merge(position_info_df, position_df[['POSITION_ID', 'POSITION_NAME']].drop_duplicates(), on='POSITION_ID')
    pos_info_sorted = pos_info_with_name.sort_values('GRADE_START_DATE')
    analysis_df = daily_work_df.sort_values('DATE')
    analysis_df = pd.merge_asof(
        analysis_df, pos_info_sorted[['EMP_ID', 'GRADE_START_DATE', 'POSITION_NAME', 'GRADE_ID']],
        left_on='DATE', right_on='GRADE_START_DATE', by='EMP_ID', direction='backward'
    )
    analysis_df = analysis_df.dropna(subset=['POSITION_NAME', 'GRADE_ID'])

    # --- 3. 계층별 데이터 집계 ---
    pos_monthly_summary = analysis_df.groupby(['POSITION_NAME', 'PAY_PERIOD'], observed=False).agg(
        TOTAL_OVERTIME_MINUTES=('OVERTIME_MINUTES', 'sum'), HEADCOUNT=('EMP_ID', 'nunique')
    ).reset_index()
    pos_monthly_summary['AVG_OVERTIME_PER_PERSON'] = (pos_monthly_summary['TOTAL_OVERTIME_MINUTES'] / pos_monthly_summary['HEADCOUNT']) / 60

    grade_monthly_summary = analysis_df.groupby(['POSITION_NAME', 'GRADE_ID', 'PAY_PERIOD'], observed=False).agg(
        TOTAL_OVERTIME_MINUTES=('OVERTIME_MINUTES', 'sum'), HEADCOUNT=('EMP_ID', 'nunique')
    ).reset_index()
    grade_monthly_summary['AVG_OVERTIME_PER_PERSON'] = (grade_monthly_summary['TOTAL_OVERTIME_MINUTES'] / grade_monthly_summary['HEADCOUNT']) / 60
    
    all_overtime_values = pd.concat([pos_monthly_summary['AVG_OVERTIME_PER_PERSON'], grade_monthly_summary['AVG_OVERTIME_PER_PERSON']])
    y_min, y_max = (all_overtime_values.min(), all_overtime_values.max()) if not all_overtime_values.empty else (0, 0)
    y_padding = (y_max - y_min) * 0.1 if (y_max - y_min) > 0 else 10
    fixed_y_range = [y_min - y_padding, y_max + y_padding]

    # --- 4. Plotly 인터랙티브 그래프 생성 ---
    fig = go.Figure()
    colors = px.colors.qualitative.Plotly
    position_filter_list = [p for p in position_order if p != 'C-Level']
    
    for i, pos_name in enumerate(position_filter_list):
        df_filtered = pos_monthly_summary[pos_monthly_summary['POSITION_NAME'] == pos_name]
        if not df_filtered.empty:
            fig.add_trace(go.Scatter(x=df_filtered['PAY_PERIOD'], y=df_filtered['AVG_OVERTIME_PER_PERSON'], mode='lines+markers', name=pos_name, line=dict(color=colors[i])))

    grade_traces_map = {}
    trace_idx_counter = len(fig.data)
    for pos_name in position_filter_list:
        grade_df = grade_monthly_summary[grade_monthly_summary['POSITION_NAME'] == pos_name]
        grades_in_pos = [g for g in grade_order if g in grade_df['GRADE_ID'].unique()]
        grade_traces_map[pos_name] = []
        for j, grade_id in enumerate(grades_in_pos):
            df_filtered = grade_df[grade_df['GRADE_ID'] == grade_id]
            if not df_filtered.empty:
                fig.add_trace(go.Scatter(x=df_filtered['PAY_PERIOD'], y=df_filtered['AVG_OVERTIME_PER_PERSON'], mode='lines+markers', name=grade_id, visible=False, line=dict(color=colors[j % len(colors)])))
                grade_traces_map[pos_name].append(trace_idx_counter)
                trace_idx_counter += 1
    
    buttons = []
    buttons.append(dict(label='전체', method='update', args=[
        {'visible': [True]*len(position_filter_list) + [False]*(len(fig.data)-len(position_filter_list))},
        {'title': '전체 직위별 월 평균 초과근무 시간 추이'}
    ]))
    for pos_name in position_filter_list:
        visibility_mask = [False] * len(fig.data)
        for trace_idx in grade_traces_map.get(pos_name, []):
            visibility_mask[trace_idx] = True
        buttons.append(dict(label=f'{pos_name}', method='update', args=[
            {'visible': visibility_mask},
            {'title': f'{pos_name} 내 직급별 월 평균 초과근무 시간 추이'}
        ]))

    fig.update_layout(
        updatemenus=[dict(active=0, buttons=buttons, direction="down", pad={"r": 10, "t": 10}, showactive=True, x=0.01, xanchor="left", y=1.1, yanchor="top")],
        title_text='직위/직급별 월 평균 1인당 초과근무 시간 드릴다운 분석',
        xaxis_title='월(YYYY-MM)', yaxis_title='1인당 평균 초과근무 (시간)',
        font_size=14, height=700,
        legend_title_text='직위/직급',
        annotations=[dict(text="직위/직급 선택:", showarrow=False, x=0, y=1.08, yref="paper", align="left")],
        xaxis_range=['2019-12', '2026-01'],
        yaxis_range=fixed_y_range
    )
    
    return fig

# 이 파일을 직접 실행할 경우 그래프를 생성하여 보여줍니다.
pio.renderers.default = 'vscode'
fig = create_figure()
fig.show()